### 비선형
- 단층 퍼셉트론은 AND, OR, NAND 게이트와 같은 구조를 갖는 모델은 구현할 수 있으나, XOR 게이트와 같이 모델이 비선형성을 필요로 할 경우 사용할 수 없음
- 비선형성을 표현하기 위해 단층 퍼셉트론을 여러 개 쌓아 hidden layer를 한 개 이상 갖는 다층 퍼셉트론 구조를 사용함
- 일반적으로 사용하는 데이터는 비선형적인 구조를 가지고 있기 때문에, activation function을 사용해서 layer의 출력값을 비선형적으로 변환함
- activation function에는 step, threshold, sigmoid, tanh, ReLU, softmax 등의 함수가 있음


### 다층 퍼셉트론 구조

In [1]:
# 다층 퍼셉트론 구조
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

# 데이터셋 클래스 선언
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x1 = df.iloc[:, 0].values
        self.x2 = df.iloc[:, 1].values
        self.y = df.iloc[:, 2].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x1[index], self.x2[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

# 모델 선언: hidden layer를 가짐
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.layer1 = nn.Sequential(
            nn.Linear(2, 2),
            nn.Sigmoid()
        )
        self.layer2 = nn.Sequential(
            nn.Linear(2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return x

# train dataset 선언
train_dataset = CustomDataset("./perceptron.csv")
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
device = "cuda" if torch.cuda.is_available() else "cpu"

# 모델, 손실 함수, optimizer 선언
model = CustomModel().to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 학습 시작: epoch 설정
for epoch in range(10000):
    cost = 0.0

    # train dataset 배치 별로 학습
    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        # 순전파
        output = model(x)
        loss = criterion(output, y)

        # 역전파 및 variable 갱신
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

# evaluation 시작
with torch.no_grad(): # 기울기 계산 비활성화
    model.eval() # 모델을 평가 모드로 변환(training과 eval 시에 다르게 동작하는 layer들을 eval 모드로 변환하기 위함)

    inputs = torch.FloatTensor([
        [0, 0],
        [0, 1],
        [1, 0],
        [1, 1]
    ]).to(device)

    outputs = model(inputs)

    print("---------")
    print(outputs)
    print(outputs <= 0.5)

<ipython-input-1-9efcddd17ea2>:18: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  x = torch.FloatTensor([self.x1[index], self.x2[index]])
<ipython-input-1-9efcddd17ea2>:19: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([self.y[index]])


Epoch : 1000, Cost : 0.692
Epoch : 2000, Cost : 0.675
Epoch : 3000, Cost : 0.237
Epoch : 4000, Cost : 0.063
Epoch : 5000, Cost : 0.033
Epoch : 6000, Cost : 0.022
Epoch : 7000, Cost : 0.017
Epoch : 8000, Cost : 0.013
Epoch : 9000, Cost : 0.011
Epoch : 10000, Cost : 0.009
---------
tensor([[0.0107],
        [0.9916],
        [0.9916],
        [0.0097]], device='cuda:0')
tensor([[ True],
        [False],
        [False],
        [ True]], device='cuda:0')
